In [1]:
import pandas as pd
import mxnet
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split


2022-11-30 10:27:12.171825: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:

#=========================================================#

##나중에 argpaser로 변경
max_grad_norm = 1
log_interval = 1000
warmup_ratio = 0.1


In [ ]:

# import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument('--num_epochs', type = int, default=5)
# parser.add_argument('--batch_size', type = int, default=16)
# parser.add_argument('--max_len', type = int, default=512)
# parser.add_argument('--learning_rate', type = float, default=5e-5)

# args = parser.parse_args()


In [3]:
EPOCHS = 10
BATCH_SIZE = 8
MAX_LEN = 512
LEARN_RATE = 5e-5

In [5]:

print(f"build model with {EPOCHS}epochs and {BATCH_SIZE}batch size")
#########################

#GPU사용
device = torch.device("cuda:1")
print(f"Using {device}")


data = pd.read_csv("../data/train.csv")
##############

print(data.info())


build model with 10epochs and 8batch size
Using cuda:1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708 entries, 0 to 1707
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1708 non-null   int64  
 1   text        1708 non-null   object 
 2   label       1708 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 40.2+ KB
None


In [7]:
data = data[['text', 'label']]

In [8]:
data

,text,label
0,[탐방-청담씨티칼리지 요양병원] 여성 암환자 명품 쉼터… 명실상부 회복공간 자리매김...,1.0
1,알록달록 이국 풍경… ‘부네치아’를 아시나요일몰·일출… 색다른 부산 여행부산 사하구...,1.0
2,용인 ‘수지 스카이뷰 푸르지오’ 4일 본보기집 개관대우건설은 경기 용인시 수지구 신...,1.0
3,교통-쇼핑-의료 ‘3박자’ 다 갖췄네[아파트 미리보기]e편한세상 청계 센트럴포레대림...,1.0
4,"‘신중동역 랜드마크 푸르지오시티’ 분양지상 49층… 오피스텔 1050실, 교통편리하...",1.0
...,...,...
1703,"머스크, 트위터 인수 후 테슬라 주식 5.4조 팔았다일론 머스크 테슬라 최고경영자(...",0.0
1704,"2008년 금융위기 예측한 마이클 버리 최근 주식 사들여교도소업체,홈쇼핑,우주기업,...",0.0
1705,"머스크, 트위터 인수 후 테슬라 주식 5.5조어치 매각했다테슬라 최고경영자(CEO)...",0.0
1706,"뉴지스탁, 미국주식 퀀트분석 '뉴지랭크US' 서비스[대구=뉴시스] 나호용 기자 = ...",1.0


In [9]:
dataset_train, dataest_test = train_test_split(data, test_size=0.1, random_state=42)


print(f"train len : {len(dataset_train)}, test len : {len(dataest_test)}")



class BERTDataset(Dataset):
    def __init__(self, dataset,bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([" ".join(dataset.iloc[i]['text'])]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i]['label']) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))



print('get bertmodel and vocab')
bertmodel, vocab = get_pytorch_kobert_model()


print("data setting")
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

train_data = BERTDataset(dataset_train, tok, MAX_LEN, True, False)
test_data = BERTDataset(dataest_test, tok, MAX_LEN, True, False)

train_dataloader = torch.utils.data.DataLoader(
    train_data, batch_size = BATCH_SIZE, num_workers = 8)

test_dataloader = torch.utils.data.DataLoader(
    test_data, batch_size = BATCH_SIZE, num_workers = 8)


class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2,   ##클래스 수 조정##
                 dr_rate = None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(
            input_ids = token_ids, token_type_ids = segment_ids.long(), 
            attention_mask = attention_mask.float().to(token_ids.device)
        )
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)



train len : 1537, test len : 171
get bertmodel and vocab
using cached model. /home/mglee/VSCODE/git_folder/article_detection/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/article_detection/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
data setting
using cached model. /home/mglee/VSCODE/git_folder/article_detection/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)


In [11]:

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=LEARN_RATE)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * EPOCHS
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader



/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:

print("Train Start")
for e in range(EPOCHS):
    train_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} / {} loss {} train acc {}".format(e+1, batch_id+1 , len(train_dataloader), loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    test_acc = 0.0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))



Train Start


  0%|          | 0/193 [00:00<?, ?it/s]

epoch 1 batch id 1 / 193 loss 0.6357103586196899 train acc 0.5
epoch 1 train acc 0.7927461139896373


  0%|          | 0/22 [00:00<?, ?it/s]

epoch 1 test acc 0.9090909090909091


...

epoch 9 test acc 0.9034090909090909


  0%|          | 0/193 [00:00<?, ?it/s]

epoch 10 batch id 1 / 193 loss 0.012393527664244175 train acc 1.0
epoch 10 train acc 0.9514248704663213


  0%|          | 0/22 [00:00<?, ?it/s]

epoch 10 test acc 0.9034090909090909


In [13]:
PATH = '../result/kobert/' 
torch.save(model, PATH + 'KoBERT_1130_e10_py.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'Kobert_1130_e10_py_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar') 

: 

In [46]:
for batch, a in train_dataloader:
    print(a)
    break

ValueError: too many values to unpack (expected 2)